In [1]:
import pandas as pd
import numpy as np
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

import json

from time import sleep

import web_crawler.scraper_walmart as walmart

# Our generated code
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%load_ext autoreload
%autoreload 2

In [2]:
step_size = 10

#with open('/libs/data.json') as f:
#    json_data = json.load(f)

products = pd.read_pickle("./data/products_pd.pickle")

products['price_dollar'] = np.nan

i=0

View the products from Walmart stores

In [14]:
products.loc[products["stores"].apply({"Migros"}.issubset or {"Monoprix"}.issubset)]

,created_t,product_name,quantity,packaging,brands,brands_tags,categories,categories_tags,origins,origins_tags,...,labels,purchase_places,stores,countries_en,main_category,energy_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,price_dollar
code,,,,,,,,,,,,,,,,,,,,,
00152297,1537293684,Mélange randonnée,200gr,sacs plastique,"Sun Queen, Migros","sun-queen,migros",,,indéterminé,indetermine,...,[],Suisse,[Migros],"[Italy, Switzerland]",,1791,,,,NaN
0039047001824,1508784068,Chocolate Chip Shortbread,175 g,,Walkers,walkers,"Sugary snacks,Biscuits and cakes,Biscuits,Cookies","en:sugary-snacks,en:biscuits-and-cakes,en:bisc...",,,...,[],,[Migros],"[France, Switzerland]",en:sugary-snacks,2167,,22,22,NaN
00840293,1465726493,maizena,250 g,,Maizena,maizena,,,,,...,[],,[Migros],[Switzerland],,,,,,NaN
08520223,1479889487,thon rosé,,,,,,,,,...,[],,[Migros],[France],,,,,,NaN
10013117,1538764583,Bifidus Drink,500 ml,,,,"Yaourts à boire,café","en:plant-based-foods-and-beverages,en:beverage...",,,...,[],Suisse,[Migros],[Switzerland],en:beverages,343,,,,NaN
12051018,1537181988,Moutarde douce,200g,tube,Migros,migros,,,,,...,[],Suisse,[Migros],[Switzerland],,410,,,,NaN
2000000014607,1421837533,Galettes de riz complet au chocolat au lait,"40g, 4 pièces",Emballage de 3 sachets de 4 galettes de riz,Migros Bio,migros-bio,"Galettes de céréales,Galettes de riz","en:plant-based-foods-and-beverages,en:plant-ba...",,,...,"[Certification Bio: Be-Bio-02, Label De Qualit...","Rolle,suisse",[Migros],[Switzerland],en:plant-based-foods-and-beverages,2080,,21,21,NaN
2000000024941,1449224765,Canga,50g,Plastic,ETİ,eti̇,"Snacks,Bar","en:sugary-snacks,en:bars,en:snacks",,,...,[],"Istanbul,Turkey",[Migros],[Turkey],en:sugary-snacks,515,,,,NaN
2000000028977,1458313508,Doğadan Ihlamur Çayı,20 Adet,Poşet,Doğadan,dogadan,,,Türkiye,turkiye,...,[Tazelik],Ankara,[Migros],[Turkey],,,,,,NaN


Request products details from Walmart Open API

In [ ]:
products.product_name = products.product_name.fillna("")
products.brands = products.brands.fillna("")

scraped_products = []

with open('walmart_products.json', mode='w', encoding='utf-8') as f:
    json.dump([], f)

    count = 0
    for i, row in products.iterrows():
        if "Walmart" in row.stores:
            count = count+1
            query = row['product_name'] + row["brands"]
            print(query.title())
            try:
                products_dicts = walmart.search(query)
            except Exception:
                continue

            for product_dict in products_dicts:
                item = {'code': row.name, 
                        'product_name': row.product_name,
                        'store_product_name': product_dict.get('name', ""),
                        'store_price': product_dict.get('salePrice', ""),
                        'store_currency': 'USD',
                        'store_category_path' : product_dict.get('categoryPath', "")
                        }
                scraped_products.append(item)
                
            # Cache intermediate results
            if not count%100:
                json.dump(scraped_products, f)
            
            # We don't wanna get banned from the server
            sleep(0.1)

    json.dump(scraped_products, f)

In [ ]:
with open('walmart_products.json', mode='w', encoding='utf-8') as f:
    json.dump(scraped_products, f)

In [ ]:
count

In [ ]:
products_dicts[0].keys()

In [ ]:
products_dicts[0]['categoryPath']